In [1]:
import torch
import pandas as pd
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cuda':
  torch.cuda.manual_seed_all(777)


In [2]:
train = pd.read_csv('../input/2021-ai-w6-p2/train.csv')
test  = pd.read_csv('../input/2021-ai-w6-p2/X_test.csv')
submit  = pd.read_csv('../input/2021-ai-w6-p2/sample_submit.csv')

In [3]:
x_train = train.drop('TARGET', axis = 1)
y_train = train['TARGET']

x_test = test.drop('ID', axis = 1)

print(x_train)

      Day  Hour  Minute  DHI  DNI   WS     RH   T
0       0     0       0    0    0  1.5  69.08 -12
1       0     0      30    0    0  1.5  69.06 -12
2       0     1       0    0    0  1.6  71.78 -12
3       0     1      30    0    0  1.6  71.75 -12
4       0     2       0    0    0  1.6  75.20 -12
...   ...   ...     ...  ...  ...  ...    ...  ..
9994  208     5       0    0    0  0.5  64.12  12
9995  208     5      30   26  383  0.8  56.18  13
9996  208     6       0   41  578  1.1  47.46  15
9997  208     6      30   52  699  1.4  44.51  17
9998  208     7       0   61  776  1.7  37.80  19

[9999 rows x 8 columns]


In [4]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()

x_train_std = scale.fit_transform(x_train)
x_test_std = scale.transform(x_test)

In [5]:
x_train = torch.FloatTensor(x_train_std)
y_train = torch.FloatTensor(y_train)
x_test = torch.FloatTensor(x_test_std)

In [6]:
layer1 = torch.nn.Linear(8,32)
layer2 = torch.nn.Linear(32,64)
layer3 = torch.nn.Linear(64,32)
layer4 = torch.nn.Linear(32,1)

sigmoid = torch.nn.Sigmoid()

In [7]:
model = torch.nn.Sequential(layer1,sigmoid,layer2,sigmoid,layer3,sigmoid,layer4).to(device)

In [8]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)


In [9]:
epochs = 10000
for epoch in range (epochs +1):
    
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    
    optimizer.zero_grad()
    
    hypothesis = model(x_train)
    
    cost = loss(hypothesis, y_train.unsqueeze(1))
    
    cost.backward()
    
    optimizer.step()
    
    if epoch % 100 == 0:
        print(epoch , cost.item())

0 1028.4976806640625
100 685.876708984375
200 682.1315307617188
300 673.57177734375
400 638.4659423828125
500 332.7088928222656
600 121.84178161621094
700 72.96858215332031
800 53.72690963745117
900 43.76524353027344
1000 36.996395111083984
1100 31.659793853759766
1200 27.400630950927734
1300 24.107210159301758
1400 21.59105110168457
1500 19.63233184814453
1600 18.051912307739258
1700 16.722179412841797
1800 15.557921409606934
1900 14.504375457763672
2000 13.527299880981445
2100 12.60622501373291
2200 11.730002403259277
2300 10.89376449584961
2400 10.096841812133789
2500 9.341365814208984
2600 8.631011962890625
2700 7.969759464263916
2800 7.36081600189209
2900 6.80594539642334
3000 6.305238246917725
3100 5.857234001159668
3200 5.45920991897583
3300 5.1075568199157715
3400 4.798110485076904
3500 4.526467323303223
3600 4.2882256507873535
3700 4.079159259796143
3800 3.8953449726104736
3900 3.733238697052002
4000 3.589693784713745
4100 3.4619669914245605
4200 3.347703456878662
4300 3.24489

In [10]:
x_test = x_test.to(device)
predict = model(x_test)
predict = predict.cpu().detach().numpy()
print(predict)

[[18.473694 ]
 [22.305014 ]
 [26.235207 ]
 ...
 [ 8.097634 ]
 [ 7.8311176]
 [ 8.057931 ]]


In [11]:
for i in range(len(predict)):
    submit['TARGET'][i] = predict[i]
print(submit)

        ID  TARGET
0        0      18
1        1      22
2        2      26
3        3      27
4        4      30
...    ...     ...
2476  2476       8
2477  2477       7
2478  2478       8
2479  2479       7
2480  2480       8

[2481 rows x 2 columns]


In [12]:
submit.to_csv('submission.csv', index = False)